In [1]:
# Program extracting first column 
import xlrd

loc = ("HPRD50_multimodal_dataset.xlsx")

wb = xlrd.open_workbook(loc)
sheet = wb.sheet_by_index(0) 
sheet.cell_value(0, 0) 
PPI = []

for i in range(sheet.nrows): 
    temp = []
    for j in range(sheet.ncols):
        temp.append(sheet.cell_value(i, j))
    PPI.append(temp)

## Vocabulary

In [2]:
import numpy as np
from gensim.parsing.preprocessing import remove_stopwords

In [3]:
corpus = []
labels = []
proteins = []
for line in PPI:
    proteins.append(line[4:6])
    corpus.append(remove_stopwords(line[1].lower()))
    labels.append(line[-1])

In [4]:
vocab = []
for doc in corpus:
    vocab += doc.split()

vocab = list(set(vocab))
values = np.array(vocab)

## Encode the Sentences / Documents

In [5]:
from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer 

In [6]:
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded.size)

1185


In [7]:
def encode_sentence(sentence):
    indices = label_encoder.transform(sentence.split())
    vector = np.zeros(integer_encoded.size, dtype = 'int')
    for index in indices:
        vector[index] = 1
    return(vector)

In [8]:
enc_corpus = []
for doc, label in zip(corpus, labels):
    enc_corpus.append(encode_sentence(doc))

In [9]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1000, svd_solver='arpack')
red_dim_corp = pca.fit_transform(enc_corpus)

# Node formatting

In [10]:
index = 283
nodes = []
for doc, label in zip(red_dim_corp, labels):
    temp = ''
    temp += str(index) + '\t'
    temp += '\t'.join([str(n) for n in doc])
    temp += '\t'+ str(float(label)) + '\n'
    index += 1
    nodes.append(temp)

In [12]:
with open('./utils/node_text', 'w') as f:
    f.writelines(nodes)

# Edge Formatting

In [13]:
protein_corpus = []
for doc_proteins in proteins:
    for protein in doc_proteins:
        protein_corpus.append(protein)
        
protein_corpus = list(set(protein_corpus))

In [14]:
protein_encoder = LabelEncoder()
protein_encodings = protein_encoder.fit_transform(protein_corpus)

In [15]:
doc_protein_encodings = []
for protein in proteins:  
    doc_protein_encodings.append(protein_encoder.transform(protein))

In [16]:
doc_protein_encodings

[array([53, 13]),
 array([ 53, 143]),
 array([ 53, 143]),
 array([143,  53]),
 array([143,  53]),
 array([53, 53]),
 array([53, 53]),
 array([ 53, 143]),
 array([ 53, 143]),
 array([53, 53]),
 array([53, 53]),
 array([ 53, 143]),
 array([53, 53]),
 array([ 53, 143]),
 array([13, 53]),
 array([13, 53]),
 array([ 13, 143]),
 array([ 13, 143]),
 array([13, 53]),
 array([13, 53]),
 array([ 13, 143]),
 array([13, 53]),
 array([ 13, 143]),
 array([53, 53]),
 array([ 53, 143]),
 array([53, 53]),
 array([53, 53]),
 array([ 53, 143]),
 array([53, 53]),
 array([ 53, 143]),
 array([ 53, 143]),
 array([ 53, 143]),
 array([53, 53]),
 array([53, 53]),
 array([ 53, 143]),
 array([53, 53]),
 array([ 53, 143]),
 array([143, 143]),
 array([143,  53]),
 array([143, 143]),
 array([143,  53]),
 array([143, 143]),
 array([143,  53]),
 array([143,  53]),
 array([143, 143]),
 array([143,  53]),
 array([143, 143]),
 array([53, 53]),
 array([53, 53]),
 array([ 53, 143]),
 array([ 53, 143]),
 array([53, 53]),
 a

In [17]:
edges = []
for i in tqdm_notebook(range(len(doc_protein_encodings))):
    for j, compare_to in enumerate(doc_protein_encodings):
        for compare in doc_protein_encodings[i]:
            if i != j and compare in compare_to:
                edges.append(str(i + 283) + "\t" + str(j + 283) + "\n")
                break

In [18]:
len(edges)

258590

In [19]:
with open('./utils/link_text', 'w') as f:
    f.writelines(edges)